# T3: Analize de Dados do SiaSUS

Forma: Trabalho em grupo com apresentação do ambiente e do código para o professor.

Objetivo:
Nesta primeira parte, que deve estar pronta para a aula do dia 03/11/2025, os grupos devem obter, preparar e carregar em um banco de dados relacional os dados do SIASUS (Sistema de Informações Ambulatoriais do SUS), disponibilizados publicamente pelo DataSUS.

Instruções:
1. 1) Obtenção da base de dados
 - Acesse o repositório do DataSUS e faça o download dos arquivos do SIASUS no formato .dbc.
 - Selecione os arquivos referentes a procedimentos ambulatoriais (ex.: PARS e outros complementares necessários para a descrição da tabela PARS).
2. 2) Conversão de formato
 - Converta os arquivos .dbc para .dbf utilizando as ferramentas recomendadas (ex.: TABWIN).
 - Certifique-se de validar a integridade dos arquivos após a conversão.
3) Visualização dos dados
 - Utilize o TABWIN, software oficial do DataSUS, para visualizar os arquivos .dbf.
 - A partir do TABWIN, gere:
  - Scripts SQL de criação de tabelas.
4) Carga dos dados em SGBD:
 - Insira os dados no banco de dados relacional de sua escolha (ex. MySQL, PostgreSQL, IBM DB2, etc).
 - Para arquivos menores, utilize diretamente os scripts de inserção gerados pelo TABWIN.
 - Para arquivos muito grandes (como o PARS, que contém milhões de registros), gere um arquivo CSV a partir do .dbf e utilize um comando de LOAD (carga em lote) no SGBD escolhido.
 - Se optar pelo MySQL (que foi o utilizado em aula):
  - Pesquise e utilize a instrução LOAD DATA na linha de comando do servidor (para evitar problemas de permissões com o cliente Workbench) conforme a documentação oficial: https://dev.mysql.com/doc/refman/8.4/en/load-data.html
Exemplo de comando LOAD DATA no terminal do servidor: LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/PARS2508.csv' INTO TABLE pars CHARACTER SET latin1 FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES  TERMINATED BY '\r\n' IGNORE 1 ROWS; 
5) A continuidade do trabalho, na aula do dia 03/11/2025, depende do ambiente do banco de dados funcionando.

## Principais libs que usaremos: 

In [4]:
# Esse projeto foi feito com o objetivo educativo para a matéria de Programção para Ciência dos dados
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt



In [5]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# /kaggle/input/sus-data-csv/CADGERRS.csv

/kaggle/input/TB_SIGTAW.csv
/kaggle/input/PARS2501.csv
/kaggle/input/tb_municip.csv
/kaggle/input/S_CID.csv
/kaggle/input/PARS2508.csv
/kaggle/input/CBO.csv
/kaggle/input/CADGERRS.csv
/kaggle/input/rl_municip_micibge.csv
/kaggle/input/PARS2505.csv
/kaggle/input/tb_micibge.csv


In [6]:
cadgerrs_data = pd.read_csv('/kaggle/input/TB_SIGTAW.csv', na_values = 'NaN', encoding = 'latin1', low_memory = False) # Atribuição de espaços vazios.
cadgerrs_data.head(n = 5)

,IP_COD,IP_DSCR
0,101010010,ATIVIDADE EDUCATIVA / ORIENTACAO EM GRUPO NA A...
1,101010028,ATIVIDADE EDUCATIVA / ORIENTACAO EM GRUPO NA A...
2,101010036,PRATICA CORPORAL / ATIVIDADE FISICA EM GRUPO
3,101010044,PRATICAS CORPORAIS EM MEDICINA TRADICIONAL CHI...
4,101010052,TERAPIA COMUNITARIA


In [7]:
# Carrega as duas tabelas
df_principal = pd.read_csv('/kaggle/input/CADGERRS.csv', na_values = 'NaN', encoding = 'latin1', low_memory = False)
df_doencas = pd.read_csv('/kaggle/input/S_CID.csv', na_values = 'NaN', encoding = 'latin1', low_memory = False)

# Junta as duas usando a coluna em comum como chave
df_completo = pd.merge(
    left=df_principal,
    right=df_doencas,
    left_on='FANTASIA',  # Nome da coluna na tabela principal
    right_on='CD_DESCR',    # Nome da coluna na tabela de doenças (pode ser diferente)
    how='left'             # Mantém todos os registros da principal, mesmo se não achar a doença
)

df_completo.head(n = 15)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,CNES,CPF_CNPJ,FANTASIA,RAZ_SOCI,RSOC_MAN,LOGRADOU,NUM_END,COMPLEME,BAIRRO,COD_CEP,...,NATUREZA,CD_COD,OPC,CAT,SUBCAT,CD_DESCR,RESTRSEXO,CAMPOS_RAD,ESTADIO,REPETE_RAD
0,7498,65174925004,RAQUEL CRISTINA DANI,CPF 651.749.250/04-RAQUEL CRISTINA DANI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7633,80924174072,BEATRIZ VALLE,CPF 809.241.740/72-BEATRIZ VALLE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13463,88212113061745,SAO JOAO FARMACIAS 29,CNPJ 88.212.113/0617-45-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13498,88212113076009,SAO JOAO FARMACIAS 36,CNPJ 88.212.113/0760-09-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14826,88212113030351,SAO JOAO FARMACIAS 14,CNPJ 88.212.113/0303-51-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,15008,88212113037011,SAO JOAO FARMACIAS 22,CNPJ 88.212.113/0370-11-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,15067,24772296000127,CLINICA VASCULAR NIENCHESKI,CNPJ 24.772.296/0001-27-CLINICA DE ANGIOLOGIA ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,15105,960472088,VANESSA ALECSSANDRA BRUNHERI PEREIRA,CPF 009.604.720/88-VANESSA ALECSSANDRA BRUNHER...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,15237,1324767030,RENATA DUTRA DE OLIVEIRA,CPF 013.247.670/30-RENATA DUTRA DE OLIVEIRA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,16640,35874383000101,CAROLINE CECCONELLO FISIOTERAPEUTA LTDA,CNPJ 35.874.383/0001-01-CAROLINE CECCONELLO FI...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Declarando arquivos:

In [8]:
import pandas as pd

# Define o caminho base para facilitar (opcional, mas limpo)
base_path = '/kaggle/input/'

# --- 1. TAREFAS FATO (Os eventos, a produção) ---
# Estas são as tabelas principais. Elas registram os eventos (atendimentos).
# Os nomes (PARS2501, PARS2505, PARS2508) sugerem que são dados mensais
# (ex: Janeiro, Maio, Agosto de 2025). Vamos carregá-los e juntá-los.

print("Carregando tabelas de produção (PARS)...")

# O que é: Tabela FATO - Produção Ambulatorial (Parte 1, ex: Mês 1)
# O que vamos contar: Registros de procedimentos, valores, local, paciente, etc.
df_prod_01 = pd.read_csv(base_path + 'PARS2501.csv', encoding='latin1', low_memory=False)

# O que é: Tabela FATO - Produção Ambulatorial (Parte 2, ex: Mês 5)
# O que vamos contar: Mais registros de procedimentos...
df_prod_05 = pd.read_csv(base_path + 'PARS2505.csv', encoding='latin1', low_memory=False)

# O que é: Tabela FATO - Produção Ambulatorial (Parte 3, ex: Mês 8)
# O que vamos contar: Mais registros de procedimentos...
df_prod_08 = pd.read_csv(base_path + 'PARS2508.csv', encoding='latin1', low_memory=False)

# --- 2. TAREFAS DIMENSÃO (Os dicionários, "de-para") ---
# Estas são as tabelas auxiliares que dão nome aos códigos das tabelas FATO.

print("Carregando tabelas de dimensão (dicionários)...")

# O que é: Tabela de Procedimentos (SIGTAP)
# O que vamos usar: Dar nome aos códigos de procedimento (ex: '0301010072' -> 'CONSULTA MEDICA')
df_procedimentos = pd.read_csv(base_path + 'TB_SIGTAW.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Municípios (IBGE)
# O que vamos usar: Dar nome aos códigos de município (ex: '431020' -> 'Ijuí').
# ESSENCIAL para a análise de fluxo de pacientes (item 4 do seu trabalho).
df_municipios = pd.read_csv(base_path + 'tb_municip.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Doenças (Classificação Internacional de Doenças - CID)
# O que vamos usar: Dar nome aos códigos de diagnóstico (ex: 'I10' -> 'Hipertensão Essencial').
df_cid = pd.read_csv(base_path + 'S_CID.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Ocupações (Classificação Brasileira de Ocupações - CBO)
# O que vamos usar: Dar nome aos códigos de ocupação do profissional
# (ex: '225125' -> 'Médico Clínico').
df_cbo = pd.read_csv(base_path + 'CBO.csv', encoding='latin1', low_memory=False)

# O que é: Cadastro de Estabelecimentos (CNES)
# O que vamos usar: Dar nome e endereço aos códigos de hospitais, clínicas e UBS
# (ex: '2254611' -> 'HOSPITAL DE CARIDADE DE IJUI').
df_estabelecimentos = pd.read_csv(base_path + 'CADGERRS.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Relação (RL) Município -> Microrregião
# O que vamos usar: Ligar um código de município a um código de microrregião.
df_rl_mun_micro = pd.read_csv(base_path + 'rl_municip_micibge.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Microrregiões (IBGE)
# O que vamos usar: Dar nome ao código da microrregião (ex: '11001' -> 'Porto Velho').
df_microrregioes = pd.read_csv(base_path + 'tb_micibge.csv', encoding='latin1', low_memory=False)

print("\nTodos os arquivos foram carregados com sucesso!")

Carregando tabelas de produção (PARS)...
Carregando tabelas de dimensão (dicionários)...

Todos os arquivos foram carregados com sucesso!


In [9]:
# Junta as 3 tabelas de produção (uma em cima da outra)
df_producao_total = pd.concat([df_prod_01, df_prod_05, df_prod_08], ignore_index=True)

# Libera a memória, já que não precisamos mais das partes separadas
del df_prod_01
del df_prod_05
del df_prod_08

print(f"Tabelas de produção combinadas!")
print(f"Total de registros para analisar: {len(df_producao_total)}")



Tabelas de produção combinadas!
Total de registros para analisar: 13210711


In [10]:
df_producao_total.head(n=10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,...,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
0,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,R,0,NaN,0.0,0.0,0.0,121003.0,NaN,3999
1,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,R,0,NaN,0.0,0.0,0.0,121003.0,NaN,3999
2,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
3,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
4,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
5,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
6,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
7,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
8,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
9,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999


In [11]:
print(df_producao_total.columns.tolist())

['PA_CODUNI', 'PA_GESTAO', 'PA_CONDIC', 'PA_UFMUN', 'PA_REGCT', 'PA_INCOUT', 'PA_INCURG', 'PA_TPUPS', 'PA_TIPPRE', 'PA_MN_IND', 'PA_CNPJCPF', 'PA_CNPJMNT', 'PA_CNPJ_CC', 'PA_MVM', 'PA_CMP', 'PA_PROC_ID', 'PA_TPFIN', 'PA_SUBFIN', 'PA_NIVCPL', 'PA_DOCORIG', 'PA_AUTORIZ', 'PA_CNSMED', 'PA_CBOCOD', 'PA_MOTSAI', 'PA_OBITO', 'PA_ENCERR', 'PA_PERMAN', 'PA_ALTA', 'PA_TRANSF', 'PA_CIDPRI', 'PA_CIDSEC', 'PA_CIDCAS', 'PA_CATEND', 'PA_IDADE', 'IDADEMIN', 'IDADEMAX', 'PA_FLIDADE', 'PA_SEXO', 'PA_RACACOR', 'PA_MUNPCN', 'PA_QTDPRO', 'PA_QTDAPR', 'PA_VALPRO', 'PA_VALAPR', 'PA_UFDIF', 'PA_MNDIF', 'PA_DIF_VAL', 'NU_VPA_TOT', 'NU_PA_TOT', 'PA_INDICA', 'PA_CODOCO', 'PA_FLQT', 'PA_FLER', 'PA_ETNIA', 'PA_VL_CF', 'PA_VL_CL', 'PA_VL_INC', 'PA_SRV_C', 'PA_INE', 'PA_NAT_JUR']


In [12]:
# A chave da nossa tabela principal (produção) é 'PA_MUNPCN'
chave_principal = 'PA_MUNPCN'

# A chave da nossa tabela auxiliar (municípios) é 'CO_MUNICIP'
chave_auxiliar = 'CO_MUNICIP'

# Converte ambas as colunas-chave para string (texto) para garantir o merge
df_producao_total[chave_principal] = df_producao_total[chave_principal].astype(str)
df_municipios[chave_auxiliar] = df_municipios[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_MUNPCN' e 'CO_MUNICIP' corrigidos para string!


In [13]:
# Vamos criar nosso dataframe de análise final, começando com este merge
# Puxamos apenas as colunas de nome e UF da tabela de municípios
df_analise = pd.merge(
    left=df_producao_total,
    right=df_municipios[['CO_MUNICIP', 'DS_NOME', 'CO_UF']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '431020')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '431020')
    how='left'                 # 'left' garante que não vamos perder nenhuma linha da principal
)

print("Merge com Municípios concluído!")

# Vamos verificar o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")

# Vamos renomear as colunas para ficar mais claro
df_analise.rename(columns={'DS_NOME': 'MUNICIPIO_PACIENTE', 'CO_UF': 'UF_PACIENTE'}, inplace=True)

# Mostra as colunas originais e as novas que foram "puxadas"
print(df_analise[['PA_MUNPCN', 'MUNICIPIO_PACIENTE', 'UF_PACIENTE']].head())

Merge com Municípios concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_MUNPCN MUNICIPIO_PACIENTE UF_PACIENTE
0    431720         Santa Rosa          43
1    430960        Horizontina          43
2    999999           Invalido          XX
3    999999           Invalido          XX
4    999999           Invalido          XX


In [14]:
# A chave da nossa tabela de análise é 'PA_PROC_ID'
chave_principal = 'PA_PROC_ID'

# A chave da nossa tabela auxiliar (procedimentos) é 'IP_COD'
chave_auxiliar = 'IP_COD'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_procedimentos[chave_auxiliar] = df_procedimentos[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_PROC_ID' e 'IP_COD' corrigidos para string!


In [15]:
# Agora fazemos o merge no 'df_analise' (que já tem os nomes dos municípios)
# Vamos puxar apenas a coluna 'IP_DSCR' (Descrição do Procedimento)
df_analise = pd.merge(
    left=df_analise,
    right=df_procedimentos[['IP_COD', 'IP_DSCR']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '0301010072')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '0301010072')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Procedimentos concluído!")

# Vamos renomear a nova coluna para ficar claro
df_analise.rename(columns={'IP_DSCR': 'NOME_PROCEDIMENTO'}, inplace=True)

# Verifica o resultado, mostrando a coluna do código e a nova coluna com o nome
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_PROC_ID', 'NOME_PROCEDIMENTO']].head())

Merge com Procedimentos concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_PROC_ID                         NOME_PROCEDIMENTO
0  206010079      TOMOGRAFIA COMPUTADORIZADA DO CRANIO
1  206020031       TOMOGRAFIA COMPUTADORIZADA DE TORAX
2  301010072  CONSULTA MEDICA EM ATENCAO ESPECIALIZADA
3  301010072  CONSULTA MEDICA EM ATENCAO ESPECIALIZADA
4  301010072  CONSULTA MEDICA EM ATENCAO ESPECIALIZADA


In [16]:
# A chave da nossa tabela de análise é 'PA_CODUNI'
chave_principal = 'PA_CODUNI'

# A chave da nossa tabela auxiliar (estabelecimentos) é 'CNES'
chave_auxiliar = 'CNES'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_estabelecimentos[chave_auxiliar] = df_estabelecimentos[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_CODUNI' e 'CNES' corrigidos para string!


In [17]:
# Vamos puxar o 'NOME FANTASIA' e a 'RAZAO SOCIAL' do estabelecimento
df_analise = pd.merge(
    left=df_analise,
    right=df_estabelecimentos[['CNES', 'FANTASIA', 'RAZ_SOCI']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '2254611')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '2254611')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Estabelecimentos concluído!")

# Renomeia as colunas para ficar mais claro
df_analise.rename(columns={
    'FANTASIA': 'NOME_ESTABELECIMENTO',
    'RAZ_SOCI': 'RAZAO_SOCIAL_ESTAB'
}, inplace=True)

# Verifica o resultado, mostrando o código e os novos nomes
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CODUNI', 'NOME_ESTABELECIMENTO']].head())

Merge com Estabelecimentos concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_CODUNI NOME_ESTABELECIMENTO
0   2254611  HOSPITAL VIDA SAUDE
1   2254611  HOSPITAL VIDA SAUDE
2   2254611  HOSPITAL VIDA SAUDE
3   2254611  HOSPITAL VIDA SAUDE
4   2254611  HOSPITAL VIDA SAUDE


In [18]:
# A chave da nossa tabela de análise é 'PA_CIDPRI'
chave_principal = 'PA_CIDPRI'

# A chave da nossa tabela auxiliar (CID) é 'CD_COD'
chave_auxiliar = 'CD_COD'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_cid[chave_auxiliar] = df_cid[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_CIDPRI' e 'CD_COD' corrigidos para string!


In [19]:
# Vamos puxar a 'CD_DESCR' (Descrição do CID)
df_analise = pd.merge(
    left=df_analise,
    right=df_cid[['CD_COD', 'CD_DESCR']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: 'I10')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: 'I10')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Diagnósticos (CID) concluído!")

# Renomeia a coluna para ficar mais claro
df_analise.rename(columns={'CD_DESCR': 'NOME_DIAGNOSTICO'}, inplace=True)

# Verifica o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CIDPRI', 'NOME_DIAGNOSTICO']].head())

Merge com Diagnósticos (CID) concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_CIDPRI   NOME_DIAGNOSTICO
0      R529       R52.9 Dor NE
1      R529       R52.9 Dor NE
2      0000  CID NAO INFORMADO
3      0000  CID NAO INFORMADO
4      0000  CID NAO INFORMADO


In [20]:
# A chave da nossa tabela de análise é 'PA_CBOCOD'
chave_principal = 'PA_CBOCOD'

# A chave da nossa tabela auxiliar (CBO) é 'CBO'
chave_auxiliar = 'CBO'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_cbo[chave_auxiliar] = df_cbo[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_CBOCOD' e 'CBO' corrigidos para string!


In [21]:
# Vamos puxar a 'DS_CBO' (Descrição do CBO)
df_analise = pd.merge(
    left=df_analise,
    right=df_cbo[['CBO', 'DS_CBO']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '225125')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '225125')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Ocupações (CBO) concluído!")

# Renomeia a coluna para ficar mais claro
df_analise.rename(columns={'DS_CBO': 'NOME_OCUPACAO'}, inplace=True)

# Verifica o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CBOCOD', 'NOME_OCUPACAO']].head())

Merge com Ocupações (CBO) concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_CBOCOD                                  NOME_OCUPACAO
0    225320  Medico em radiologia e diagnostico por imagem
1    225320  Medico em radiologia e diagnostico por imagem
2    225203                    Medico em cirurgia vascular
3    225203                    Medico em cirurgia vascular
4    225203                    Medico em cirurgia vascular


# Microregioes por paciente: 

In [22]:
# A chave da nossa tabela de análise é 'PA_MUNPCN'
chave_principal = 'PA_MUNPCN'

# A chave da nossa tabela "ponte" (RL) é 'CO_MUNICIP'
chave_auxiliar = 'CO_MUNICIP'

# Converte ambas as colunas-chave para string (texto)
# df_analise[chave_principal] já é string, mas vamos garantir
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_rl_mun_micro[chave_auxiliar] = df_rl_mun_micro[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_MUNPCN' e 'CO_MUNICIP' corrigidos para string!


## Merge A (puxar apenas a coluna 'CO_MICIBGE' da tabela "ponte")

In [23]:
# Vamos puxar apenas a coluna 'CO_MICIBGE' da tabela "ponte"
df_analise = pd.merge(
    left=df_analise,
    right=df_rl_mun_micro[['CO_MUNICIP', 'CO_MICIBGE']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '431020')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '431020')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Relação de Microrregião concluído!")
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_MUNPCN', 'MUNICIPIO_PACIENTE', 'CO_MICIBGE']].head())

Merge com Relação de Microrregião concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_MUNPCN MUNICIPIO_PACIENTE  CO_MICIBGE
0    431720         Santa Rosa     43900.0
1    430960        Horizontina     43900.0
2    999999           Invalido         NaN
3    999999           Invalido         NaN
4    999999           Invalido         NaN


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


# Microregiao em relacao ao nome

In [24]:
# A nova chave da nossa tabela de análise é 'CO_MICIBGE'
chave_principal = 'CO_MICIBGE'

# A chave da nossa tabela final (microrregiões) é 'CO_MICIBGE'
chave_auxiliar = 'CO_MICIBGE'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_microrregioes[chave_auxiliar] = df_microrregioes[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'CO_MICIBGE' e 'CO_MICIBGE' corrigidos para string!


# Merge B (puxar a 'DS_NOME' (nome da microrregião))

In [25]:
# Vamos puxar a 'DS_NOME' (nome da microrregião)
df_analise = pd.merge(
    left=df_analise,
    right=df_microrregioes[['CO_MICIBGE', 'DS_NOME']], 
    left_on=chave_principal,
    right_on=chave_auxiliar,
    how='left'
)

print("Merge com Nomes de Microrregião concluído!")

# Renomeia a coluna para ficar claro (DS_NOME já existe por causa do merge dos municípios)
# O Pandas pode ter criado 'DS_NOME_x' e 'DS_NOME_y'. Vamos renomear a última
# Se o merge anterior já renomeou, este nome pode ser diferente.
# Vamos assumir que a nova coluna é 'DS_NOME' da df_microrregioes e renomeá-la
df_analise.rename(columns={'DS_NOME': 'NOME_MICRORREGIAO'}, inplace=True)

# Verifica o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['MUNICIPIO_PACIENTE', 'CO_MICIBGE', 'NOME_MICRORREGIAO']].head())

Merge com Nomes de Microrregião concluído!

Verificando as 5 primeiras linhas do resultado:
  MUNICIPIO_PACIENTE CO_MICIBGE NOME_MICRORREGIAO
0         Santa Rosa    43900.0               NaN
1        Horizontina    43900.0               NaN
2           Invalido        nan               NaN
3           Invalido        nan               NaN
4           Invalido        nan               NaN


# Començando a Análize de Dados: